In [ ]:
!pip install PyMuPDF
!pip install tdqm

  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1323 sha256=ea17cbed854074357534487569ab9c80cecfc9b330edffce6f5d2be07f10ccd4
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm


In [ ]:
!pip install tqdm


In [ ]:
pdf_path = "human-nutrition-text.pdf"

In [ ]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text:str) -> str:
  """Perform minor formatting on text."""
  cleaned_text = text.replace("\n", " ").strip()
  return cleaned_text
def open_and_read_pdf(pdf_path: str) -> list[dict]:
  """Open and read a PDF file."""
  doc = fitz.open(pdf_path)
  pages_and_texts = []
  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text=text)
    pages_and_texts.append({"page_number": page_number - 41,
                            "page_char_count": len(text),
                            "page_word_count": len(text.split(" ")),
                            "page_sentence_count": len(text.split(".")),
                            "page_token_count": len(text)/4,
                            "text": text,
                            })

  return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [ ]:
import random
random.sample(pages_and_texts, k=4)

[{'page_number': 1132,
  'page_char_count': 1376,
  'page_word_count': 237,
  'page_sentence_count': 10,
  'page_token_count': 344.0,
  'text': 'Food Insecurity  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  Addressing Hunger  Government agencies also play an important role in addressing  hunger via federal food-assistance programs. The agencies provide  debit cards (formerly distributed in the form of food vouchers or  food stamps) to consumers to help them purchase food and they  also provide other forms of aid to low-income adults and families  who face hunger and nutritional deficits. This topic will be  discussed in greater detail later in this chapter.  Hunger relates to appetite and is the body’s response to a need for  nourishment. Through stomach discomfort or intestinal rumbling,  the body alerts the brain that it requires food. This uneasy sensation  is easily addressed with a snack or a full meal. However, the term  “h

### Text Processing

Splitting pages into sentences specifically 10 sentencess
# Using spaCy

In [ ]:
!pip install spacy

In [ ]:
from spacy.lang.en import English

nlp = English()

nlp.add_pipe('sentencizer')

doc = nlp("This is a sentence. this another sentence. I like Elephants")
assert len(list(doc.sents))==3

list(doc.sents)

[This is a sentence., this another sentence., I like Elephants]

In [ ]:
for item in tqdm(pages_and_texts):
  item["sentences"] = list(nlp(item["text"]).sents)

  #check if all sentences are string or not
  item["sentences"] = [str(sentence) for sentence in item["sentences"]]

  item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [ ]:
random.sample(pages_and_texts, k=1)

[{'page_number': 492,
  'page_char_count': 1059,
  'page_word_count': 190,
  'page_sentence_count': 7,
  'page_token_count': 264.75,
  'text': 'Photo by  Martins  Zemlickis on  unsplash.co m / CC0  https:/ /unspl ash.com/ photos/ NPFu4GfFZ7 E  Factors Affecting Energy  Expenditure  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  Physiological and Genetic Influences  Why is it so difficult for some people to lose weight and for others  to gain weight? One theory is that every person has a “set point”  of energy balance. This set point can also be called a fat-stat or  lipostat, meaning the brain senses body fatness and triggers  changes in energy intake or expenditure to maintain body fatness  within a target range. Some believe that this theory provides an  explanation as to why after dieting, most people return to their  original weight not long after stopping the diet. Another theory is  referred to as the “settling” point system,

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,page_sentence_count_spacy
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.59,198.89,14.18,287.15,10.32
std,348.86,560.44,95.75,9.54,140.11,6.30
min,-41.00,0.00,1.00,1.00,0.00,0.00
25%,260.75,762.75,134.00,8.00,190.69,5.00
50%,562.50,1232.50,215.00,13.00,308.12,10.00
75%,864.25,1605.25,271.25,19.00,401.31,15.00
max,1166.00,2308.00,429.00,82.00,577.00,28.00


In [ ]:
 #define split size to turn groups of sentence into chunks
 num_sentences_chunk_size = 10

 #creating a function to split lists of texts recursively into chunk size
 #e.g [30] -> [10,10,10]
 def split_list(input_list: list[str], slice_size: int = num_sentences_chunk_size) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]
 test_list = list(range(25))
 split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [ ]:
for item in tqdm(pages_and_texts):
  item['sentence_chunks'] = split_list(input_list=item['sentences'],
                                             slice_size = num_sentences_chunk_size)
  item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [ ]:
random.sample(pages_and_texts, k=1)

[{'page_number': 418,
  'page_char_count': 1718,
  'page_word_count': 291,
  'page_sentence_count': 24,
  'page_token_count': 429.5,
  'text': 'sources of lysine. Following a vegetarian diet and getting the  recommended protein intake is also made a little more difficult  because the digestibility of plant-based protein sources is lower  than the digestibility of animal-based protein.  To begin planning a more plant-based diet, start by finding out  which types of food you want to eat and in what amounts you should  eat them to ensure that you get the protein you need. The Dietary  Guidelines Advisory Committee (DGAC) has analyzed how three  different, plant-based dietary patterns can meet the recommended  dietary guidelines for all nutrients.2  The diets are defined in the following manner:  • Plant-based. Fifty percent of protein is obtained from plant  foods.  • Lacto-ovo vegetarian. All animal products except eggs and  dairy are eliminated.  • Vegan. All animal products are elimina

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.59,198.89,14.18,287.15,10.32,1.53
std,348.86,560.44,95.75,9.54,140.11,6.30,0.64
min,-41.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,260.75,762.75,134.00,8.00,190.69,5.00,1.00
50%,562.50,1232.50,215.00,13.00,308.12,10.00,1.00
75%,864.25,1605.25,271.25,19.00,401.31,15.00,2.00
max,1166.00,2308.00,429.00,82.00,577.00,28.00,3.00


# Spilitting each chunk into its own item
We will embed each chunk of sentences into its own numerical representation

In [ ]:
import re

pages_and_chunks = []

for item in tqdm(pages_and_texts):
  for sentence_chunk in item["sentence_chunks"]:
    chunk_dict = {}
    chunk_dict["page_number"] = item["page_number"]

    #join the sentences together into a paragraph-like structure aka join list of sentences into one paragraph
    joined_sentence_chunk = "".join(sentence_chunk).replace(" ", " ").strip()
    joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)

    chunk_dict["sentence_chunk"] = joined_sentence_chunk

    #Get Some Stats on our chunks
    chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
    chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
    chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = -4 chars

    pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)



  0%|          | 0/1208 [00:00<?, ?it/s]

1843

In [ ]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 847,
  'sentence_chunk': 'not only by the kinds of liquids given to an infant, but also by  the frequency and length of time that fluids are given. Giving a  child a bottle of juice or other sweet liquids several times each  day, or letting a baby suck on a bottle longer than a mealtime,  either when awake or asleep, can also cause early childhood caries. In addition, this practice affects the development and position of  the teeth and the jaw. The risk of early childhood caries continues  into the toddler years as children begin to consume more foods  with a high sugar content. Therefore, parents should avoid putting  their children to bed with a bottle, and giving their children sugary  snacks and beverages. If a parent insists on giving their child a bottle  in bed, then it should be filled with water only. Infancy  |  847',
  'chunk_char_count': 811,
  'chunk_word_count': 155,
  'chunk_token_count': 202.75}]

In [ ]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,583.38,750.40,128.29,187.60
std,347.79,455.59,79.84,113.90
min,-41.00,14.00,4.00,3.50
25%,280.50,321.50,51.50,80.38
50%,586.00,764.00,131.00,191.00
75%,890.00,1138.00,194.00,284.50
max,1166.00,1870.00,412.00,467.50


In [ ]:
## 1.56 has been done